In [49]:
import cv2
import numpy as np
import pyautogui
from tensorflow import keras 
from ultralytics import YOLO

In [50]:
# Load your YOLO model
yolo_path = 'Yolos/Insects/train41/weights/best.pt'  
bee_path = '../Classifiers/classifiers/beeOrNot.keras'
wasp_path = '../Classifiers/classifiers/waspOrNot.keras'
yolo_model = YOLO('yolov10n.pt')
bee_model = keras.models.load_model(bee_path)
wasp_model = keras.models.load_model(wasp_path)

In [51]:
# Set the detection threshold
threshold = 0.2
input_size = (224,224)

In [52]:
# Get screen dimensions
screen_width, screen_height = pyautogui.size()

# Define region for screen capture (full screen)
region = (0, 0, screen_width, screen_height)

# Create an OpenCV window
window_name = "YOLO Detection with Custom Classification"
cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
cv2.resizeWindow(window_name, 800, 600)

while True:
    # Capture the screen
    screenshot = pyautogui.screenshot(region=region)

    # Convert screenshot to a NumPy array (PIL to NumPy)
    frame = np.array(screenshot)

    # Convert from RGB (PIL format) to BGR (OpenCV format)
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    # Run YOLO model for object detection
    results = yolo_model.predict(source=frame, conf=threshold, verbose=False)

    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()  # YOLO bounding boxes
        confidences = result.boxes.conf.cpu().numpy()  # YOLO confidence scores


        for i, box in enumerate(boxes):
            x1, y1, x2, y2 = map(int, box)

            # Clip bounding box coordinates to frame dimensions
            x1 = max(0, min(x1, frame.shape[1] - 1))
            y1 = max(0, min(y1, frame.shape[0] - 1))
            x2 = max(0, min(x2, frame.shape[1] - 1))
            y2 = max(0, min(y2, frame.shape[0] - 1))

            # Ensure the bounding box is valid
            if x2 > x1 and y2 > y1:
                # Crop the detected region
                cropped_region = frame[y1:y2, x1:x2]
                
                # Resize the cropped region to match your model's input size
                cropped_input = cv2.resize(cropped_region, input_size)
                cropped_input = cv2.cvtColor(cropped_input, cv2.COLOR_RGB2BGR)
                cropped_input = np.expand_dims(cropped_input, axis=0)  # Add batch dimension
                # Run classification using your custom model
                predictions = bee_model.predict(cropped_input, verbose=False)
                predicted_class = "Bee"
                confidence_score = np.max(predictions)
                if confidence_score < 0.7:
                    predictions = wasp_model.predict(cropped_input, verbose=False)
                    predicted_class = "Wasp"
                    confidence_score = np.max(predictions)
                    if confidence_score < 0.7:
                        predicted_class = "Other"
                        confidence_score = 1.0 - confidence_score
                    
                    

                # Draw bounding box and label
                label = f"Class {predicted_class} ({confidence_score:.2f})"
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)  # Draw rectangle
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)  # Add label

    # Display the frame with detections and classifications
    cv2.imshow(window_name, frame)

    # Exit when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cv2.destroyAllWindows()